# Support Vector Machines (SVMs)

Author: Jacob McCabe

### Overview

This notebook will take a look at Support Vector Machines for the purpose of classification. The main ideas covered will include:

1. The main idea behind SVMs
2. Hard Margin Problems
3. Soft Margin Problems
4. Kernel Functions
5. Example using `SVC` from `skikit-learn`

### What is a SVM?

Support Vector Machines are a category of non-probabilistic, supervised learning algorithms for binary classification. They work by fitting a hyperplane between classes, and are trained by finding the hyperplane that maximizes the margin. Margin is the distance between the decision boundary and closest data point. Those points that are closest to the decision boundary and thus define the margin are referred to as support vectors.

![A visualization of margin and support vectors.](images\svm_visualization.png)


One of the important properties of this class of algorithms is that they are equivalent to solving a convex optimization problem, so any local solution is also a global optimum. Before getting to the objective function, let's define things mathematically.

Recall that the goal is to find the hyperplane representing the decision boundary


### Hard Margin Problems



### Soft Margin Problems



### Kernel Functions



### Example using `SVC` from `skikit-learn`

